In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import calendar

# Importing the data

In [181]:
#lead and dtc level scores
lead_dtc_scores = pd.read_excel("C:\\Users\\skbandha\\OneDrive - Lowe's Companies Inc\\Projects\\PSE Leads\Mar_May_2017.xlsx")
lead_dtc_scores.head(10)

,avg_survival,avg_cumhaz,dtc,date_lead
0,0.981331,0.018871,0,2017-03-01
1,0.973561,0.026843,1,2017-03-01
2,0.964657,0.036067,2,2017-03-01
3,0.953434,0.047829,3,2017-03-01
4,0.941074,0.060961,4,2017-03-01
5,0.929387,0.073556,5,2017-03-01
6,0.915226,0.089054,6,2017-03-01
7,0.897893,0.108385,7,2017-03-01
8,0.881872,0.126625,8,2017-03-01
9,0.866037,0.145013,9,2017-03-01


In [182]:
#Leads converted
lead_actual_converted = pd.read_excel("C:\\Users\\skbandha\\OneDrive - Lowe's Companies Inc\\Projects\\PSE Leads\Lead_Converted_17_18_PSE.xlsx")
lead_actual_converted.head(10)

,converts,date_lead,date_sold
0,1,2017-01-01,2017-01-02
1,1,2017-01-01,2017-01-04
2,3,2017-01-01,2017-01-05
3,6,2017-01-01,2017-01-06
4,4,2017-01-01,2017-01-09
5,4,2017-01-01,2017-01-10
6,3,2017-01-01,2017-01-11
7,3,2017-01-01,2017-01-12
8,3,2017-01-01,2017-01-13
9,1,2017-01-01,2017-01-14


In [183]:
#Number of leads generated
leads_generated_2018 = pd.read_excel("C:\\Users\\skbandha\\OneDrive - Lowe's Companies Inc\\Projects\\PSE Leads\Lead_gen_18.xlsx")
print("Sample", leads_generated_2018.head(10))
print("Minimmum Date", min(leads_generated_2018.date_lead))
print("Maximum Date", max(leads_generated_2018.date_lead))

Sample    lead_gen  date_lead
0       792 2018-01-01
1      1716 2018-01-02
2      1402 2018-01-03
3      1344 2018-01-04
4      1532 2018-01-05
5      1351 2018-01-06
6       821 2018-01-07
7      1589 2018-01-08
8      1545 2018-01-09
9      1493 2018-01-10
Minimmum Date 2018-01-01 00:00:00
Maximum Date 2018-08-31 00:00:00


In [184]:
#Number of leads generated - Mar to May 17
leads_generated_2017 = pd.read_excel("C:\\Users\\skbandha\\OneDrive - Lowe's Companies Inc\\Projects\\PSE Leads\Leads_Generated_Mar_May17.xlsx")
print("Sample", leads_generated_2017.head(10))
print("Minimmum Date", min(leads_generated_2017.date_lead))
print("Maximum Date", max(leads_generated_2017.date_lead))

Sample    date_lead  lead_gen
0 2017-03-01      1993
1 2017-03-02      2142
2 2017-03-03      2256
3 2017-03-04      2605
4 2017-03-05      1545
5 2017-03-06      2648
6 2017-03-07      2346
7 2017-03-08      2169
8 2017-03-09      2588
9 2017-03-10      2230
Minimmum Date 2017-03-01 00:00:00
Maximum Date 2017-05-31 00:00:00


In [185]:
#DTC Survival probabilities
survival_probs_dtc = pd.read_excel("C:\\Users\\skbandha\\OneDrive - Lowe's Companies Inc\\Projects\\PSE Leads\DTC_Surv_Probs.xlsx")
#rename columns
survival_probs_dtc.rename(columns = {'St':'avg_survival', 'Cum_haz':'avg_cumhaz'}, inplace = True)

print("Sample", survival_probs_dtc.head(10))

Sample    dtc  avg_survival  avg_cumhaz
0    0      0.984163    0.015988
1    1      0.977496    0.022806
2    2      0.969845    0.030696
3    3      0.960152    0.040793
4    4      0.949368    0.052159
5    5      0.939184    0.063030
6    6      0.926809    0.076418
7    7      0.911627    0.093115
8    8      0.897419    0.109019
9    9      0.883391    0.125000


In [198]:
#Survival Probabilities from Feb to Aug '18
probs_feb_aug18 = pd.read_excel("C:\\Users\\skbandha\\OneDrive - Lowe's Companies Inc\\Projects\\PSE Leads\Feb_aug18.xlsx")
print(probs_feb_aug18.head(10))

   avg_survival  avg_cumhaz  dtc  date_lead
0      0.982498    0.017695    0 2018-02-01
1      0.982382    0.017813    0 2018-02-02
2      0.981174    0.019039    0 2018-02-03
3      0.982565    0.017629    0 2018-02-04
4      0.982398    0.017796    0 2018-02-05
5      0.982578    0.017615    0 2018-02-06
6      0.982812    0.017375    0 2018-02-07
7      0.981876    0.018326    0 2018-02-08
8      0.982081    0.018119    0 2018-02-09
9      0.981244    0.018968    0 2018-02-10


# Building for Mar to May 2017 data

# Data Preperation

In [186]:
#Assigning attributes
lead_probs_28day = lead_dtc_scores
dtc_surv_probs = survival_probs_dtc
leads_number = leads_generated_2017
actuals = lead_actual_converted

In [187]:
#Creating one source for all probabilities
dist_lead_dates = pd.DataFrame(lead_probs_28day['date_lead'].unique()).reset_index()
#print(list(dist_lead_dates.columns.values))
dist_lead_dates.rename(columns = {0:'date_lead'}, inplace = True)

cols_to_drop = dist_lead_dates['index']
dist_lead_dates = dist_lead_dates[['date_lead']]
#print("Sample",dist_lead_dates.head(10))

#Cartesian product of dtc and lead dates
dist_lead_dates['key'] = 0
dtc_surv_probs['key'] = 0

probs_29_49 = dist_lead_dates.merge(dtc_surv_probs, how = 'left', on = 'key')
#print(probs_29_49.groupby(['date_lead']).size())

#Calculating marginal probabilities
probs_29_49.sort_values(by = ['date_lead', 'dtc'])
probs_29_49['lag'] = probs_29_49.groupby(['date_lead'])['avg_cumhaz'].shift(1)
#print(probs_29_49.head(20))

probs_29_49['marginal_prob'] = np.where(probs_29_49['dtc'] == 0, probs_29_49['avg_cumhaz'], probs_29_49['avg_cumhaz'] - probs_29_49['lag'])
#print(probs_29_49.head(60))

lead_probs_28day.sort_values(by = ['date_lead', 'dtc'])
lead_probs_28day['lag'] = lead_probs_28day.groupby(['date_lead'])['avg_cumhaz'].shift(1)
#print(probs_29_49.head(20))

lead_probs_28day['marginal_prob'] = np.where(lead_probs_28day['dtc'] == 0, lead_probs_28day['avg_cumhaz'], lead_probs_28day['avg_cumhaz'] - lead_probs_28day['lag'])
#print(lead_probs_28day.head(60))

#Restricting the probabilities to only 29 to 49 probabilities only
probs_29_49 = probs_29_49[(probs_29_49['dtc'] <= 49) & (probs_29_49['dtc'] >= 29)]
#print(probs_29_49.groupby(['date_lead']).size())

#Appending the 28 day forecast with 29 forecast
final_probs = lead_probs_28day.append(probs_29_49, ignore_index = True)
final_probs = final_probs.drop(['key'], axis = 1)
final_probs.sort_values(by = ['date_lead', 'dtc'])
print("Frequency by Date",final_probs.groupby(['date_lead']).size())
print(final_probs.head(60))

Frequency by Date date_lead
2017-03-01    50
2017-03-02    50
2017-03-03    50
2017-03-04    50
2017-03-05    50
2017-03-06    50
2017-03-07    50
2017-03-08    50
2017-03-09    50
2017-03-10    50
2017-03-11    50
2017-03-12    50
2017-03-13    50
2017-03-14    50
2017-03-15    50
2017-03-16    50
2017-03-17    50
2017-03-18    50
2017-03-19    50
2017-03-20    50
2017-03-21    50
2017-03-22    50
2017-03-23    50
2017-03-24    50
2017-03-25    50
2017-03-26    50
2017-03-27    50
2017-03-28    50
2017-03-29    50
2017-03-30    50
              ..
2017-05-02    50
2017-05-03    50
2017-05-04    50
2017-05-05    50
2017-05-06    50
2017-05-07    50
2017-05-08    50
2017-05-09    50
2017-05-10    50
2017-05-11    50
2017-05-12    50
2017-05-13    50
2017-05-14    50
2017-05-15    50
2017-05-16    50
2017-05-17    50
2017-05-18    50
2017-05-19    50
2017-05-20    50
2017-05-21    50
2017-05-22    50
2017-05-23    50
2017-05-24    50
2017-05-25    50
2017-05-26    50
2017-05-27    50
201

In [188]:
#Merging with number of leads
final_probs = final_probs.merge(leads_number, how = 'left', on = 'date_lead')
final_probs.head(10)

,avg_cumhaz,avg_survival,date_lead,dtc,lag,marginal_prob,lead_gen
0,0.018871,0.981331,2017-03-01,0,NaN,0.018871,1993
1,0.026843,0.973561,2017-03-01,1,0.018871,0.007972,1993
2,0.036067,0.964657,2017-03-01,2,0.026843,0.009224,1993
3,0.047829,0.953434,2017-03-01,3,0.036067,0.011762,1993
4,0.060961,0.941074,2017-03-01,4,0.047829,0.013131,1993
5,0.073556,0.929387,2017-03-01,5,0.060961,0.012595,1993
6,0.089054,0.915226,2017-03-01,6,0.073556,0.015498,1993
7,0.108385,0.897893,2017-03-01,7,0.089054,0.019331,1993
8,0.126625,0.881872,2017-03-01,8,0.108385,0.018240,1993
9,0.145013,0.866037,2017-03-01,9,0.126625,0.018389,1993


In [189]:
#Matching with actuals

#Create date_sold
final_probs['date_sold'] = final_probs['date_lead']
final_probs['date_sold'] += pd.to_timedelta(final_probs['dtc'], unit = 'D')
final_probs.head(10)

print(final_probs.dtypes)
print("")
print(actuals.dtypes)
print(actuals.head(10))

final_probs = final_probs.merge(actuals, how = 'left', on = ['date_lead', 'date_sold'])

avg_cumhaz              float64
avg_survival            float64
date_lead        datetime64[ns]
dtc                       int64
lag                     float64
marginal_prob           float64
lead_gen                  int64
date_sold        datetime64[ns]
dtype: object

converts              int64
date_lead    datetime64[ns]
date_sold    datetime64[ns]
dtype: object
   converts  date_lead  date_sold
0         1 2017-01-01 2017-01-02
1         1 2017-01-01 2017-01-04
2         3 2017-01-01 2017-01-05
3         6 2017-01-01 2017-01-06
4         4 2017-01-01 2017-01-09
5         4 2017-01-01 2017-01-10
6         3 2017-01-01 2017-01-11
7         3 2017-01-01 2017-01-12
8         3 2017-01-01 2017-01-13
9         1 2017-01-01 2017-01-14


In [190]:
#Predicted probability
final_probs['predicted_converts'] = final_probs['avg_survival'] * final_probs['marginal_prob'] * final_probs['lead_gen']
print(final_probs.head(10))

   avg_cumhaz  avg_survival  date_lead  dtc       lag  marginal_prob  \
0    0.018871      0.981331 2017-03-01    0       NaN       0.018871   
1    0.026843      0.973561 2017-03-01    1  0.018871       0.007972   
2    0.036067      0.964657 2017-03-01    2  0.026843       0.009224   
3    0.047829      0.953434 2017-03-01    3  0.036067       0.011762   
4    0.060961      0.941074 2017-03-01    4  0.047829       0.013131   
5    0.073556      0.929387 2017-03-01    5  0.060961       0.012595   
6    0.089054      0.915226 2017-03-01    6  0.073556       0.015498   
7    0.108385      0.897893 2017-03-01    7  0.089054       0.019331   
8    0.126625      0.881872 2017-03-01    8  0.108385       0.018240   
9    0.145013      0.866037 2017-03-01    9  0.126625       0.018389   

   lead_gen  date_sold  converts  predicted_converts  
0      1993 2017-03-01      24.0           36.907294  
1      1993 2017-03-02      20.0           15.468968  
2      1993 2017-03-03      31.0          

In [191]:
#Creating weekdays
final_probs['dtc_weekday'] = final_probs['date_sold'].dt.weekday_name
print(final_probs.head(10))

   avg_cumhaz  avg_survival  date_lead  dtc       lag  marginal_prob  \
0    0.018871      0.981331 2017-03-01    0       NaN       0.018871   
1    0.026843      0.973561 2017-03-01    1  0.018871       0.007972   
2    0.036067      0.964657 2017-03-01    2  0.026843       0.009224   
3    0.047829      0.953434 2017-03-01    3  0.036067       0.011762   
4    0.060961      0.941074 2017-03-01    4  0.047829       0.013131   
5    0.073556      0.929387 2017-03-01    5  0.060961       0.012595   
6    0.089054      0.915226 2017-03-01    6  0.073556       0.015498   
7    0.108385      0.897893 2017-03-01    7  0.089054       0.019331   
8    0.126625      0.881872 2017-03-01    8  0.108385       0.018240   
9    0.145013      0.866037 2017-03-01    9  0.126625       0.018389   

   lead_gen  date_sold  converts  predicted_converts dtc_weekday  
0      1993 2017-03-01      24.0           36.907294   Wednesday  
1      1993 2017-03-02      20.0           15.468968    Thursday  
2     

In [192]:
#Creating dependent variable
final_probs['error'] = (final_probs['converts'] - final_probs['predicted_converts']) / final_probs['predicted_converts']
print(final_probs.head(10))

   avg_cumhaz  avg_survival  date_lead  dtc       lag  marginal_prob  \
0    0.018871      0.981331 2017-03-01    0       NaN       0.018871   
1    0.026843      0.973561 2017-03-01    1  0.018871       0.007972   
2    0.036067      0.964657 2017-03-01    2  0.026843       0.009224   
3    0.047829      0.953434 2017-03-01    3  0.036067       0.011762   
4    0.060961      0.941074 2017-03-01    4  0.047829       0.013131   
5    0.073556      0.929387 2017-03-01    5  0.060961       0.012595   
6    0.089054      0.915226 2017-03-01    6  0.073556       0.015498   
7    0.108385      0.897893 2017-03-01    7  0.089054       0.019331   
8    0.126625      0.881872 2017-03-01    8  0.108385       0.018240   
9    0.145013      0.866037 2017-03-01    9  0.126625       0.018389   

   lead_gen  date_sold  converts  predicted_converts dtc_weekday     error  
0      1993 2017-03-01      24.0           36.907294   Wednesday -0.349722  
1      1993 2017-03-02      20.0           15.468968 

# Building the model - correct predictions

In [193]:
import statsmodels.formula.api as smf
import statsmodels.api as sm

reg = smf.ols(formula = "error~dtc_weekday + dtc", data = final_probs).fit()
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  error   R-squared:                       0.669
Model:                            OLS   Adj. R-squared:                  0.668
Method:                 Least Squares   F-statistic:                     1206.
Date:                Sat, 29 Sep 2018   Prob (F-statistic):               0.00
Time:                        22:44:10   Log-Likelihood:                 241.41
No. Observations:                4193   AIC:                            -466.8
Df Residuals:                    4185   BIC:                            -416.1
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                    0.3301      0.011     30.501      0.000       0.309       0.351
dtc_weekday[T.Monday]       -0.3706      0.013    -29.187      0.000      -0.396      -0.346
dtc_weekday[T.Saturday]     -0.6210      0.013    -48.285      0.000      -0.646      -0.596
dtc_weekday[T.Sunday]       -0.9312      0.015    -61.390      0.000      -0.961      -0.901
dtc_weekday[T.Thursday]     -0.3215      0.013    -25.322      0.000      -0.346      -0.297
dtc_weekday[T.Tuesday]      -0.4055      0.013    -31.958      0.000      -0.430      -0.381
dtc_weekday[T.Wednesday]    -0.4981      0.013    -39.102      0.000      -0.523      -0.473
dtc                         -0.0167      0.000    -66.709      0.000      -0.017      -0.016
==============================================================================
Omnibus:                      445.512   Durbin-Watson:                   1.719
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              986.142
Skew:                           0.651   Prob(JB):                    7.28e-215
Kurtosis:                       4.987   Cond. No.                         199.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [197]:
final_scores = final_probs
final_scores['weekday_coeff'] = np.where(final_scores['dtc_weekday'] == 'Monday', -0.3706,
                                        np.where(final_scores['dtc_weekday'] == 'Tuesday', -0.4055,
                                                np.where(final_scores['dtc_weekday'] == 'Wednesday', -0.4981,
                                                         np.where(final_scores['dtc_weekday'] == 'Thursday', -0.3215,
                                                                  np.where(final_scores['dtc_weekday'] == 'Saturday', -0.6210,
                                                                           np.where(final_scores['dtc_weekday'] == 'Sunday', -0.9312,0
                                                                                   ))))))

final_scores['pred_errors'] = 0.2998 + final_scores['weekday_coeff'] + (-0.0167 * final_scores['dtc'])
final_scores['new_pred_converts'] = (1+final_scores['pred_errors']) * final_probs['predicted_converts']

predicts = pd.DataFrame(final_scores.groupby(final_scores['dtc'])['new_pred_converts'].sum()).reset_index()
actuals = pd.DataFrame(final_scores.groupby(final_scores['dtc'])['converts'].sum()).reset_index()
final = predicts.merge(actuals, how = 'left', on = 'dtc')
final['errors'] = abs(final['new_pred_converts'] - final['converts'])/final['converts']
print(final)


    dtc  new_pred_converts  converts    errors
0     0        3486.463821    3479.0  0.002145
1     1        1387.957833    1486.0  0.065977
2     2        1574.193156    1699.0  0.073459
3     3        1969.701375    2043.0  0.035878
4     4        2117.260038    2185.0  0.031002
5     5        1872.904883    2119.0  0.116137
6     6        2313.897238    2589.0  0.106258
7     7        2884.660934    2948.0  0.021485
8     8        2520.777480    2817.0  0.105155
9     9        2457.307770    2671.0  0.080005
10   10        2484.768444    2687.0  0.075263
11   11        2354.207715    2506.0  0.060572
12   12        1916.400920    2144.0  0.106156
13   13        2179.419898    2347.0  0.071402
14   14        2350.668974    2411.0  0.025023
15   15        1908.032549    1990.0  0.041190
16   16        1795.789457    1885.0  0.047327
17   17        1776.897366    1726.0  0.029489
18   18        1593.305912    1677.0  0.049907
19   19        1265.086981    1331.0  0.049521
20   20      

# Validating for Feb to Aug Leads

# Data Prep

In [199]:
#Assigning attributes
lead_probs_28day = probs_feb_aug18
dtc_surv_probs = survival_probs_dtc
leads_number = leads_generated_2018
actuals = lead_actual_converted

In [204]:
#Creating one source for all probabilities
dist_lead_dates = pd.DataFrame(lead_probs_28day['date_lead'].unique()).reset_index()
#print(list(dist_lead_dates.columns.values))
dist_lead_dates.rename(columns = {0:'date_lead'}, inplace = True)

cols_to_drop = dist_lead_dates['index']
dist_lead_dates = dist_lead_dates[['date_lead']]
#print("Sample",dist_lead_dates.head(10))

#Cartesian product of dtc and lead dates
dist_lead_dates['key'] = 0
dtc_surv_probs['key'] = 0

probs_29_49 = dist_lead_dates.merge(dtc_surv_probs, how = 'left', on = 'key')
#print(probs_29_49.groupby(['date_lead']).size())

#Calculating marginal probabilities
probs_29_49 = probs_29_49.sort_values(by = ['date_lead', 'dtc'])
probs_29_49['lag'] = probs_29_49.groupby(['date_lead'])['avg_cumhaz'].shift(1)
#print(probs_29_49.head(20))

probs_29_49['marginal_prob'] = np.where(probs_29_49['dtc'] == 0, probs_29_49['avg_cumhaz'], probs_29_49['avg_cumhaz'] - probs_29_49['lag'])
#print(probs_29_49.head(60))

lead_probs_28day = lead_probs_28day[(lead_probs_28day['dtc'] <= 28)]
lead_probs_28day = lead_probs_28day.sort_values(by = ['date_lead', 'dtc'])
lead_probs_28day['lag'] = lead_probs_28day.groupby(['date_lead'])['avg_cumhaz'].shift(1)
print(lead_probs_28day.head(40))

lead_probs_28day['marginal_prob'] = np.where(lead_probs_28day['dtc'] == 0, lead_probs_28day['avg_cumhaz'], lead_probs_28day['avg_cumhaz'] - lead_probs_28day['lag'])
#print(lead_probs_28day.head(60))

#Restricting the probabilities to only 29 to 49 probabilities only
probs_29_49 = probs_29_49[(probs_29_49['dtc'] <= 49) & (probs_29_49['dtc'] >= 29)]
#print(probs_29_49.groupby(['date_lead']).size())

#Appending the 28 day forecast with 29 forecast
final_probs = lead_probs_28day.append(probs_29_49, ignore_index = True)
final_probs = final_probs.drop(['key'], axis = 1)
final_probs.sort_values(by = ['date_lead', 'dtc'])
print("Frequency by Date",final_probs.groupby(['date_lead']).size())
print(final_probs.head(60))

      avg_survival  avg_cumhaz  dtc  date_lead       lag  marginal_prob
0         0.982498    0.017695    0 2018-02-01       NaN       0.017695
212       0.975056    0.025331    1 2018-02-01  0.017695       0.007635
424       0.966577    0.034114    2 2018-02-01  0.025331       0.008783
636       0.955719    0.045488    3 2018-02-01  0.034114       0.011375
848       0.943469    0.058490    4 2018-02-01  0.045488       0.013002
1060      0.932021    0.070819    5 2018-02-01  0.058490       0.012329
1272      0.918038    0.086110    6 2018-02-01  0.070819       0.015291
1484      0.901003    0.105091    7 2018-02-01  0.086110       0.018981
1696      0.884848    0.123446    8 2018-02-01  0.105091       0.018355
1908      0.868956    0.141874    9 2018-02-01  0.123446       0.018428
2120      0.852652    0.161166   10 2018-02-01  0.141874       0.019292
2332      0.836468    0.180714   11 2018-02-01  0.161166       0.019548
2544      0.822299    0.198191   12 2018-02-01  0.180714       0

In [205]:
#Merging with number of leads
final_probs = final_probs.merge(leads_number, how = 'left', on = 'date_lead')
final_probs.head(10)

,avg_cumhaz,avg_survival,date_lead,dtc,lag,marginal_prob,lead_gen
0,0.017695,0.982498,2018-02-01,0,NaN,0.017695,1676
1,0.025331,0.975056,2018-02-01,1,0.017695,0.007635,1676
2,0.034114,0.966577,2018-02-01,2,0.025331,0.008783,1676
3,0.045488,0.955719,2018-02-01,3,0.034114,0.011375,1676
4,0.058490,0.943469,2018-02-01,4,0.045488,0.013002,1676
5,0.070819,0.932021,2018-02-01,5,0.058490,0.012329,1676
6,0.086110,0.918038,2018-02-01,6,0.070819,0.015291,1676
7,0.105091,0.901003,2018-02-01,7,0.086110,0.018981,1676
8,0.123446,0.884848,2018-02-01,8,0.105091,0.018355,1676
9,0.141874,0.868956,2018-02-01,9,0.123446,0.018428,1676


In [213]:
#Matching with actuals

#Create date_sold
final_probs['date_sold'] = final_probs['date_lead']
final_probs['date_sold'] += pd.to_timedelta(final_probs['dtc'], unit = 'D')
final_probs.head(10)

print(final_probs.dtypes)
print("")
print(actuals.dtypes)
print(actuals.head(10))

print(max(final_probs['date_lead']))

final_probs = final_probs.merge(actuals, how = 'left', on = ['date_lead', 'date_sold'])

avg_cumhaz                   float64
avg_survival                 float64
date_lead             datetime64[ns]
dtc                            int64
lag                          float64
marginal_prob                float64
lead_gen                       int64
date_sold             datetime64[ns]
converts                     float64
predicted_converts           float64
dtc_weekday                   object
weekday_coeff                float64
pred_errors                  float64
new_pred_converts            float64
dtype: object

dtc           int64
converts    float64
dtype: object
   dtc  converts
0    0    4529.0
1    1    1788.0
2    2    2112.0
3    3    2585.0
4    4    2784.0
5    5    2688.0
6    6    3227.0
7    7    3813.0
8    8    3570.0
9    9    3341.0
2018-08-31 00:00:00


KeyError: 'date_lead'

In [207]:
#Predicted probability
final_probs['predicted_converts'] = final_probs['avg_survival'] * final_probs['marginal_prob'] * final_probs['lead_gen']
print(final_probs.head(10))

   avg_cumhaz  avg_survival  date_lead  dtc       lag  marginal_prob  \
0    0.017695      0.982498 2018-02-01    0       NaN       0.017695   
1    0.025331      0.975056 2018-02-01    1  0.017695       0.007635   
2    0.034114      0.966577 2018-02-01    2  0.025331       0.008783   
3    0.045488      0.955719 2018-02-01    3  0.034114       0.011375   
4    0.058490      0.943469 2018-02-01    4  0.045488       0.013002   
5    0.070819      0.932021 2018-02-01    5  0.058490       0.012329   
6    0.086110      0.918038 2018-02-01    6  0.070819       0.015291   
7    0.105091      0.901003 2018-02-01    7  0.086110       0.018981   
8    0.123446      0.884848 2018-02-01    8  0.105091       0.018355   
9    0.141874      0.868956 2018-02-01    9  0.123446       0.018428   

   lead_gen  date_sold  converts  predicted_converts  
0      1676 2018-02-01      39.0           29.138104  
1      1676 2018-02-02      23.0           12.477572  
2      1676 2018-02-03      10.0          

In [208]:
#Creating weekdays
final_probs['dtc_weekday'] = final_probs['date_sold'].dt.weekday_name
print(final_probs.head(10))

   avg_cumhaz  avg_survival  date_lead  dtc       lag  marginal_prob  \
0    0.017695      0.982498 2018-02-01    0       NaN       0.017695   
1    0.025331      0.975056 2018-02-01    1  0.017695       0.007635   
2    0.034114      0.966577 2018-02-01    2  0.025331       0.008783   
3    0.045488      0.955719 2018-02-01    3  0.034114       0.011375   
4    0.058490      0.943469 2018-02-01    4  0.045488       0.013002   
5    0.070819      0.932021 2018-02-01    5  0.058490       0.012329   
6    0.086110      0.918038 2018-02-01    6  0.070819       0.015291   
7    0.105091      0.901003 2018-02-01    7  0.086110       0.018981   
8    0.123446      0.884848 2018-02-01    8  0.105091       0.018355   
9    0.141874      0.868956 2018-02-01    9  0.123446       0.018428   

   lead_gen  date_sold  converts  predicted_converts dtc_weekday  
0      1676 2018-02-01      39.0           29.138104    Thursday  
1      1676 2018-02-02      23.0           12.477572      Friday  
2     

In [230]:
final_scores = final_probs
final_scores['weekday_coeff'] = np.where(final_scores['dtc_weekday'] == 'Monday', -0.3706,
                                        np.where(final_scores['dtc_weekday'] == 'Tuesday', -0.4055,
                                                np.where(final_scores['dtc_weekday'] == 'Wednesday', -0.4981,
                                                         np.where(final_scores['dtc_weekday'] == 'Thursday', -0.3215,
                                                                  np.where(final_scores['dtc_weekday'] == 'Saturday', -0.6210,
                                                                           np.where(final_scores['dtc_weekday'] == 'Sunday', -0.9312,0
                                                                                   ))))))

final_scores['pred_errors'] = 0.2998 + final_scores['weekday_coeff'] + (-0.0167 * final_scores['dtc'])
final_scores['new_pred_converts'] = (1+final_scores['pred_errors']) * final_probs['predicted_converts']
final_scores = final_scores[(final_scores['date_sold'] <= '2018-07-01') & (final_scores['date_lead'] <= '2018-05-31')]
final_scores['month'] = final_scores['date_lead'].dt.month
print(final_scores.head(10))

print(final_scores.groupby(['date_sold'])['converts'].sum())

predicts = pd.DataFrame(final_scores.groupby(['dtc'])['new_pred_converts'].sum()).reset_index()
actuals = pd.DataFrame(final_scores.groupby(['dtc'])['converts'].sum()).reset_index()
print(predicts.head(10))
print("")
print(actuals.head(10))
final = predicts.merge(actuals, how = 'left', on = ['dtc'])
final['errors'] = abs(final['new_pred_converts'] - final['converts'])/final['converts']
print(final)

final.to_excel("C:\\Users\\skbandha\\OneDrive - Lowe's Companies Inc\\Projects\\PSE Leads\\Final_Errors.xlsx", sheet_name='sheet1', index=False)


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


   avg_cumhaz  avg_survival  date_lead  dtc       lag  marginal_prob  \
0    0.017695      0.982498 2018-02-01    0       NaN       0.017695   
1    0.025331      0.975056 2018-02-01    1  0.017695       0.007635   
2    0.034114      0.966577 2018-02-01    2  0.025331       0.008783   
3    0.045488      0.955719 2018-02-01    3  0.034114       0.011375   
4    0.058490      0.943469 2018-02-01    4  0.045488       0.013002   
5    0.070819      0.932021 2018-02-01    5  0.058490       0.012329   
6    0.086110      0.918038 2018-02-01    6  0.070819       0.015291   
7    0.105091      0.901003 2018-02-01    7  0.086110       0.018981   
8    0.123446      0.884848 2018-02-01    8  0.105091       0.018355   
9    0.141874      0.868956 2018-02-01    9  0.123446       0.018428   

   lead_gen  date_sold  converts  predicted_converts dtc_weekday  \
0      1676 2018-02-01      39.0           29.138104    Thursday   
1      1676 2018-02-02      23.0           12.477572      Friday   
2  